<a href="https://colab.research.google.com/github/kundajelab/label_shift_experiments/blob/master/Download_CIFAR10_models_from_zenodo_and_make_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget

In [2]:
import keras
from keras.models import load_model
from keras.models import Model
import wget
import os
import sys
from keras.datasets import cifar10
import numpy as np
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train, axis=(0, 1, 2, 3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
x_valid = x_train[:10000]
y_valid = y_train[:10000]

model_files = [
    "model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-10_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-20_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-30_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-40_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-50_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-60_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-70_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-80_bestbefore-100_currentepoch-100_valacc-90_vgg.h5",
    "model_cifar10_balanced_seed-90_bestbefore-100_currentepoch-100_valacc-91_vgg.h5"
]

for model_file in model_files:
  print("On model", model_file)
  if (os.path.isfile(model_file)==False):
    print("Downloading", model_file)
    wget.download("https://sandbox.zenodo.org/record/274839/files/"
                  +model_file+"?download=1", out=model_file)
  model = load_model(model_file)
  
  pre_softmax_model = Model(input=model.input,
                            output=model.layers[-2].output)
  print("Making predictions on validation set")
  valid_preacts = pre_softmax_model.predict(x_valid)
  print("Making predictions on test set")
  test_preacts = pre_softmax_model.predict(x_test)
  
  print("Test accuracy",np.mean(np.argmax(test_preacts,axis=1)
                                == np.squeeze(y_test)))
  print("Valid accuracy",np.mean(np.argmax(valid_preacts,axis=1)
                                 == np.squeeze(y_valid)))
  sys.stdout.flush()
  test_predictions_file = ("testpreacts_"+model_file.split(".")[0])+".txt"
  print("Saving", test_predictions_file)
  f = open(test_predictions_file,'w')
  for test_preact in test_preacts:
     f.write("\t".join([str(x) for x in test_preact])+"\n") 
  f.close()
  !gzip $test_predictions_file

  valid_predictions_file = ("validpreacts_"+model_file.split(".")[0])+".txt"
  print("Saving", valid_predictions_file)
  f = open(valid_predictions_file,'w')
  for valid_preact in valid_preacts:
     f.write("\t".join([str(x) for x in valid_preact])+"\n") 
  f.close()
  !gzip $valid_predictions_file
  

Using TensorFlow backend.


On model model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.h5
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("co..., outputs=Tensor("de...)`


Making predictions on validation set
Making predictions on test set
Test accuracy 0.8967
Valid accuracy 0.9054
Saving testpreacts_model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.txt
Saving validpreacts_model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.txt
On model model_cifar10_balanced_seed-100_bestbefore-100_currentepoch-100_valacc-91_vgg.h5
Making predictions on validation set
Making predictions on test set
Test accuracy 0.9007
Valid accuracy 0.91
Saving testpreacts_model_cifar10_balanced_seed-100_bestbefore-100_currentepoch-100_valacc-91_vgg.txt
Saving validpreacts_model_cifar10_balanced_seed-100_bestbefore-100_currentepoch-100_valacc-91_vgg.txt
On model model_cifar10_balanced_seed-10_bestbefore-100_currentepoch-100_valacc-91_vgg.h5
Making predictions on validation set
Making predictions on test set
Test accuracy 0.9029
Valid accuracy 0.9073
Saving testpreacts_model_cifar10_balanced_seed-10_bestbefore-100_currentepoch-100_valacc

In [3]:
!md5sum *.gz

19909f2352b9b642c2c53eda6be99c6e  testpreacts_model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.txt.gz
944c099b71af05c5cae874d406a40434  testpreacts_model_cifar10_balanced_seed-100_bestbefore-100_currentepoch-100_valacc-91_vgg.txt.gz
a16528de6a4cf028f896f5fe97ba4ca4  testpreacts_model_cifar10_balanced_seed-10_bestbefore-100_currentepoch-100_valacc-91_vgg.txt.gz
2756fc2454c6f04bd43a70a7beb09771  testpreacts_model_cifar10_balanced_seed-20_bestbefore-100_currentepoch-100_valacc-91_vgg.txt.gz
c801f07ae28b225e867f46d69e61caa2  testpreacts_model_cifar10_balanced_seed-30_bestbefore-100_currentepoch-100_valacc-91_vgg.txt.gz
67550d5f91cb446e3cea13906e3c2311  testpreacts_model_cifar10_balanced_seed-40_bestbefore-100_currentepoch-100_valacc-91_vgg.txt.gz
a4ab01e8159c0d7a641a889840639f89  testpreacts_model_cifar10_balanced_seed-50_bestbefore-100_currentepoch-100_valacc-91_vgg.txt.gz
6d4d78bc3c65797e1c8d888c80da81d0  testpreacts_model_cifar10_balanced_seed-60_bestbefore-10

In [0]:
from google.colab import files
import glob

for file in glob.glob("*.gz"):
  files.download(file)